In [1]:
from visualizer import tree_to_array
from skbio import TreeNode
import numpy as np
import collections
import json

In [2]:
t = TreeNode.read('/home/evan/data/gg_13_8_otus/trees/99_otus_unannotated.tree')
t.name = "root"

In [3]:
for c in t.children:
    c.length = 0

In [4]:
def tree_annotated(tree):
    queue = collections.deque([tree])
    idx = 0
    while queue:
        node = queue.pop()

        distance_to_longest_tip = 0
        for tip in node.tips():
            distance_to_longest_tip = max(distance_to_longest_tip,
                                          node.distance(tip))
        node.d = distance_to_longest_tip
        node.i = idx
        
        queue.extendleft(node.children)
        idx += 1

def tree_grp(tree, depth):
    queue = collections.deque([tree])
    mapping = {}
    while queue:
        node = queue.pop()
        if node.d <= depth:
            if node.children:
                mapping[node.i] = [n.name for n in node.tips()]
            else:
                mapping[node.i] = [node.name]
        else:    
            queue.extendleft(node.children)
    return mapping

In [5]:
x = tree_to_array(t)

In [77]:
with open('/tmp/phylo.json') as fh:
    x = json.load(fh)

In [6]:
def collapse(x, depth):
    search = collections.deque()
    
    children = x['children']
    distances = x['distances']
    names = x['names']
    length = len(children)
    
    mapping = {}
    
    for i in range(length):
        if search:
            g_start, g_end, g = search[-1]
            if g_start <= i < g_end:
                if children[i]:
                    end = length
                    for k in range(i+1, length):
                        if children[k]:
                            end = children[k]
                            break
                    
                    search.appendleft([children[i], end, g])
                else:
                    g.append(names[i])
                    
                if i + 1 == g_end:
                    search.pop()
                
            else:
                if children[i]:
                    if distances[i] <= depth:
                        end = length
                        for k in range(i+1, length):
                            if children[k]:
                                end = children[k]
                                break

                        g = []
                        mapping[i] = g
                        search.appendleft([children[i], end, g])
                else:
                    mapping[i] = [names[i]]
            
        elif children[i]:
            if distances[i] <= depth:
                end = length
                for k in range(i+1, length):
                    if children[k]:
                        end = children[k]
                        break

                g = []
                mapping[i] = g
                search.appendleft([children[i], end, g])
        else:
            mapping[i] = [names[i]]
            
    return mapping
            

In [7]:
for item, _ in zip(list(zip([n.ljust(10) for n in x['names']], range(len(x['names'])), x['children'], x['parent'], x['distances'])), range(100)):
    print('\t'.join(map(str, item)))

root      	0	1	0	1.29407
          	1	3	0	0.69088
          	2	5	0	1.29407
          	3	7	1	0.67876
52        	4	0	1	0
          	5	9	2	0.35599
          	6	11	2	1.1920300000000001
          	7	13	3	0.5991899999999999
          	8	15	3	0.65196
          	9	17	5	0.30291999999999997
203503    	10	0	5	0
          	11	19	6	0.39924
          	12	21	6	1.1686800000000002
          	13	23	7	0.42098000000000013
          	14	25	7	0.5806399999999999
          	15	27	8	0.31443000000000004
          	16	29	8	0.6003499999999999
          	17	31	9	0.29369999999999996
          	18	33	9	0.11652
          	19	35	11	0.34658
          	20	37	11	0.38248
          	21	39	12	0.20293
          	22	41	12	1.15806
          	23	43	13	0.4004700000000001
          	24	45	13	0.08861
          	25	47	14	0.5665499999999999
          	26	49	14	0.52307
          	27	51	15	0.22236000000000003
          	28	53	15	0.2715
          	29	55	16	0.31484
          	30	57	16	0.53452
          	31	59	17	0.27701
1873000   	32	0	

In [8]:
with open('/tmp/phylo-gg.json', 'w') as fh:
    fh.write(json.dumps({'children': x['children'], 'names': x['names'], 'distances': x['distances']}, separators=(',', ':')))

In [9]:
full_list = []
def collapse_2(x, depth):
    global full_list
    full_list = []

    search = collections.deque([None])
    
    children = x['children']
    distances = x['distances']
    names = x['names']
    length = len(children)
    
    mapping = collections.OrderedDict()
    
    start, end, g = 0, 0, []
    pending_start = 0
    pending_g = []
    for i, child in enumerate(children):        
        # finalize last iteration
        if pending_start and child:
            search.appendleft([pending_start, child, pending_g])
            pending_start = 0
        if i == end:
            search.pop()
        if search:
            start, end, g = search[-1]
        
        # current iteration
        if child:
            if start <= i < end:
                pending_g = g
                pending_start = child
            elif distances[i] <= depth:
                pending_g = []
                mapping[i] = pending_g
                pending_start = child
            else:
                pass
        else:
            if start <= i < end:
                g.append(i)
                full_list.append(i)
            elif pending_start and pending_start <= i:
                pending_g.append(i)
                full_list.append(i)
            else:
                mapping[i] = [i]
                full_list.append(i)
            
    return mapping
            

In [10]:
%timeit tree_grp(t, 0)

AttributeError: 'TreeNode' object has no attribute 'd'

In [11]:
%timeit collapse(x, 0)

208 ms ± 4.57 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%timeit collapse_2(x, 0)

239 ms ± 2.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%timeit tree_grp(t, 1)

AttributeError: 'TreeNode' object has no attribute 'd'

In [14]:
%timeit collapse(x, 1)

327 ms ± 3.38 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%timeit collapse_2(x, 1)

210 ms ± 7.36 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
breaks = list(np.unique(x['distances']))
breaks

[0.0,
 0.00013999999999999999,
 0.00014999999999999999,
 0.00016000000000000001,
 0.00018000000000000001,
 0.00020000000000000001,
 0.00025000000000000001,
 0.00025999999999999998,
 0.00026999999999999995,
 0.00027999999999999998,
 0.00029,
 0.00029999999999999997,
 0.00036000000000000002,
 0.00039999999999999996,
 0.00040999999999999999,
 0.00041999999999999996,
 0.00042000000000000002,
 0.00042999999999999999,
 0.00044000000000000002,
 0.00044999999999999999,
 0.00046999999999999999,
 0.00048000000000000001,
 0.00050000000000000001,
 0.00051000000000000004,
 0.00051999999999999995,
 0.00052000000000000006,
 0.00052999999999999998,
 0.00055000000000000003,
 0.00055999999999999995,
 0.00056999999999999998,
 0.00058,
 0.00059000000000000003,
 0.00059999999999999995,
 0.00060999999999999997,
 0.00062,
 0.00063000000000000003,
 0.00064000000000000005,
 0.00064999999999999997,
 0.00066,
 0.00067000000000000002,
 0.00068000000000000005,
 0.00068999999999999997,
 0.00069999999999999999,
 0.0

In [17]:
len(breaks)

35725

In [18]:
for e in collapse_2(x, 2.5).items():
    print("%02d: %r" % e)

00: [4, 10, 32, 33, 58, 61, 62, 65, 67, 68, 71, 72, 81, 83, 86, 91, 95, 97, 98, 106, 109, 116, 117, 121, 123, 124, 125, 127, 130, 131, 135, 139, 140, 143, 159, 161, 164, 167, 169, 170, 171, 172, 173, 179, 180, 181, 187, 191, 194, 197, 198, 199, 200, 201, 202, 205, 216, 217, 219, 221, 222, 229, 235, 237, 239, 240, 241, 244, 246, 249, 251, 252, 253, 255, 256, 257, 258, 259, 261, 262, 265, 266, 267, 268, 269, 271, 273, 275, 281, 287, 289, 290, 291, 293, 295, 299, 311, 317, 319, 321, 322, 323, 325, 333, 334, 337, 338, 339, 340, 347, 348, 351, 352, 363, 364, 365, 366, 369, 371, 373, 374, 375, 376, 377, 379, 380, 389, 393, 397, 398, 399, 401, 409, 413, 415, 424, 425, 431, 432, 435, 439, 445, 446, 455, 463, 464, 471, 472, 473, 475, 483, 489, 491, 499, 500, 502, 507, 509, 513, 523, 528, 529, 530, 531, 533, 535, 539, 541, 544, 549, 555, 562, 563, 566, 571, 575, 576, 578, 585, 586, 593, 596, 597, 598, 605, 606, 611, 612, 613, 619, 620, 623, 627, 635, 642, 643, 644, 645, 648, 650, 653, 655, 657, 

In [19]:
full_list

[4,
 10,
 32,
 33,
 58,
 61,
 62,
 65,
 67,
 68,
 71,
 72,
 81,
 83,
 86,
 91,
 95,
 97,
 98,
 106,
 109,
 116,
 117,
 121,
 123,
 124,
 125,
 127,
 130,
 131,
 135,
 139,
 140,
 143,
 159,
 161,
 164,
 167,
 169,
 170,
 171,
 172,
 173,
 179,
 180,
 181,
 187,
 191,
 194,
 197,
 198,
 199,
 200,
 201,
 202,
 205,
 216,
 217,
 219,
 221,
 222,
 229,
 235,
 237,
 239,
 240,
 241,
 244,
 246,
 249,
 251,
 252,
 253,
 255,
 256,
 257,
 258,
 259,
 261,
 262,
 265,
 266,
 267,
 268,
 269,
 271,
 273,
 275,
 281,
 287,
 289,
 290,
 291,
 293,
 295,
 299,
 311,
 317,
 319,
 321,
 322,
 323,
 325,
 333,
 334,
 337,
 338,
 339,
 340,
 347,
 348,
 351,
 352,
 363,
 364,
 365,
 366,
 369,
 371,
 373,
 374,
 375,
 376,
 377,
 379,
 380,
 389,
 393,
 397,
 398,
 399,
 401,
 409,
 413,
 415,
 424,
 425,
 431,
 432,
 435,
 439,
 445,
 446,
 455,
 463,
 464,
 471,
 472,
 473,
 475,
 483,
 489,
 491,
 499,
 500,
 502,
 507,
 509,
 513,
 523,
 528,
 529,
 530,
 531,
 533,
 535,
 539,
 541,
 544,
 549,


In [20]:
tree_annotated(t)

In [65]:
%timeit [tree_grp(t, b) for b in breaks[-10:]]

4.06 s ± 17.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [64]:
%timeit [collapse(x, b) for b in breaks[-10:]]

3.35 s ± 24.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [67]:
%timeit [collapse(x, b) for b in breaks[:10]]

1.88 s ± 7.72 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [63]:
%timeit [collapse_2(x, b) for b in breaks[-10:]]

1.93 s ± 30.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [66]:
%timeit [collapse_2(x, b) for b in breaks[:10]]

1.69 s ± 19.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
